In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var trainDataPath = "../datasets/iris-train.txt";
var testDataPath = "../datasets/iris-test.txt";

In [ ]:
public class IrisData
{
    [LoadColumn(0)]
    public float Label;

    [LoadColumn(1, 4)]
    public float Features { get; set; }
}

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var trainingDataView = mlContext.Data.LoadFromTextFile<IrisData>(trainDataPath, hasHeader: true);
var testDataView = mlContext.Data.LoadFromTextFile<IrisData>(testDataPath, hasHeader: true);

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "KeyColumn", inputColumnName: nameof(IrisData.Label))
                         .Append(mlContext.Transforms.Concatenate("Features", nameof(IrisData.Features)).AppendCacheCheckpoint(mlContext))
                         .Append(mlContext.Transforms.NormalizeMinMax("Features", "Features"));

In [ ]:
var trainer = mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy(labelColumnName: "KeyColumn", featureColumnName: "Features")
              .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: nameof(IrisData.Label), inputColumnName: "KeyColumn"));

var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
var trainedModel = trainingPipeline.Fit(trainingDataView);

In [ ]:
var predictions = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions);

In [ ]:
Console.WriteLine($"Micro Accuracy: {metrics.MicroAccuracy:P2}");
Console.WriteLine($"Macro Accuracy: {metrics.MacroAccuracy:P2}");
Console.WriteLine($"Log Loss: {metrics.LogLoss:P2}");
Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());

Micro Accuracy: 100,00%
Macro Accuracy: 100,00%
Log Loss: 3,51%

Confusion table
          ||========================
PREDICTED ||     0 |     1 |     2 | Recall
TRUTH     ||========================
        0 ||     9 |     0 |     0 | 1,0000
        1 ||     0 |    10 |     0 | 1,0000
        2 ||     0 |     0 |    10 | 1,0000
          ||========================
Precision ||1,0000 |1,0000 |1,0000 |

